In [0]:
df_customer = spark.read.format("delta").table("stream1.stream1.customers_bronze")
df_customers = df_customer.dropDuplicates(["customer_id"]).drop("source_file", "ingestion_time").withColumn(
    "full_name", F.concat_ws(" ", F.col("first_name"), F.col("last_name"))
)
df_customers.dropna()
display(df_customers)
df_customers.printSchema()

In [0]:
df_order = spark.read.format("delta").table("stream1.stream1.orders_bronze")
df_orders = df_order.dropDuplicates(["order_id"]).drop("source_file", "ingestion_time").dropna(subset=["order_id", "wallet_id"])
display(df_orders)
df_orders.printSchema()

In [0]:
df_price = spark.read.format("delta").table("stream1.stream1.prices_bronze")
df_price = df_price.dropDuplicates(["price_id", "trade_id"]).select("asset_name", "price_id", "price_value", "timestamp", "trade_id")
display(df_price)
df_price.printSchema()


In [0]:
df_1 = (
    spark.read.format("delta").table("stream1.stream1.trades_bronze")
    .dropDuplicates(["order_id", "trade_id"])
    .drop("source_file", "ingestion_time")
)
display(df_1)
df_1.printSchema()

In [0]:
df_wallets = (
    spark.read.format("delta").table("stream1.stream1.wallets_bronze")
    .dropDuplicates(["wallet_id"])
    .dropna(subset=["wallet_id", "customer_id"])
    .drop("ingestion_time", "source_file")
)
display(df_wallets)
df_wallets.printSchema()


In [0]:
df_customers.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.customers_silver")
df_wallets.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.wallets_silver")
df_orders.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.orders_silver")
df_1.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.trades_silver")
df_price.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.prices_silver")

test

In [0]:
customers_silver = spark.read.format("delta").table("stream1.stream1.customers_silver")
wallets_silver = spark.read.format("delta").table("stream1.stream1.wallets_silver")
orders_silver = spark.read.format("delta").table("stream1.stream1.orders_silver")
trades_silver = spark.read.format("delta").table("stream1.stream1.trades_silver")
prices_silver = spark.read.format("delta").table("stream1.stream1.prices_silver")

In [0]:
gold_base = (
    customers_silver.alias("c")
    .join(wallets_silver.alias("w"), F.col("c.customer_id") == F.col("w.customer_id"), "inner")
    .join(orders_silver.alias("o"), F.col("w.wallet_id") == F.col("o.wallet_id"), "inner")
    .join(trades_silver.alias("t"), F.col("o.order_id") == F.col("t.order_id"), "inner")
    .join(prices_silver.alias("p"), F.col("t.trade_id") == F.col("p.trade_id"), "inner")
    .select(
        F.col("c.customer_id"),
        F.col("c.full_name"),
        F.col("c.email"),
        F.col("c.phone"),
        F.col("c.address"),
        F.col("c.kyc_type"),
        F.col("c.kyc_details"),
        F.col("w.wallet_id"),
        F.col("w.currency"),
        F.col("w.wallet_balance").cast("double").alias("wallet_balance"),
        F.col("o.order_id"),
        F.col("o.order_type"),
        F.col("o.asset"),
        F.col("o.price").cast("double").alias("order_price"),
        F.col("o.quantity").cast("double").alias("order_quantity"),
        F.col("o.order_status"),
        F.col("t.trade_id"),
        F.col("t.trade_type"),
        F.col("t.trade_value").cast("double").alias("trade_value"),
        F.col("t.trade_fee").cast("double").alias("trade_fee"),
        F.col("t.trade_date"),
        F.col("p.asset_name"),
        F.col("p.price_value").cast("double").alias("price_value"),
        F.col("p.timestamp").alias("price_timestamp")
    )
)
display(gold_base)
gold_base.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.base_for_gold")

In [0]:
from pyspark.sql import functions as F

# CUSTOMERS
df_customers0 = (
    spark.read.format("delta").table("stream1.stream1.customers_bronze")
    .dropDuplicates(["customer_id"])
    .dropna(subset=["customer_id"])
    .withColumn("full_name", F.concat_ws(" ", "first_name", "last_name"))
    .withColumn("created_at", F.to_timestamp("created_at"))
)

# WALLETS
df_wallets0 = (
    spark.read.format("delta").table("stream1.stream1.wallets_bronze")
    .dropDuplicates(["wallet_id"])
    .dropna(subset=["wallet_id", "customer_id"])
    .withColumn("created_at", F.to_timestamp("created_at"))
)

# ORDERS
df_orders0 = (
    spark.read.format("delta").table("stream1.stream1.orders_bronze")
    .dropDuplicates(["order_id"])
    .dropna(subset=["order_id", "wallet_id"])
    .withColumn("created_at", F.to_timestamp("created_at"))
    .withColumn("updated_at", F.to_timestamp("updated_at"))
)

# TRADES
df_trades0 = (
    spark.read.format("delta").table("stream1.stream1.trades_bronze")
    .dropDuplicates(["trade_id"])
    .dropna(subset=["trade_id", "order_id"])
    .withColumn("trade_date", F.to_timestamp("trade_date"))
)

# PRICES
df_prices0 = (
    spark.read.format("delta").table("stream1.stream1.prices_bronze")
    .dropDuplicates(["price_id"])
    .dropna(subset=["price_id", "trade_id"])
    .withColumn("timestamp", F.to_timestamp("timestamp"))
)
display(df_customers0)
display(df_wallets0)
display(df_orders0)
display(df_trades0)
display(df_prices0)